<a href="https://colab.research.google.com/github/Altaieb-Mohammed/lab_2corse/blob/master/lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#---------------------------------------------------------- Лабораторная работа №4:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def load_data(url):
    try:
        df = pd.read_csv(url)
        print("Данные успешно загружены из URL.")
    except Exception as e:
        print(f"Ошибка загрузки данных: {e}")
        from google.colab import files
        uploaded = files.upload()
        df = pd.read_csv(next(iter(uploaded)))
        print("Данные успешно загружены из локального файла.")
    return df

url = "https://raw.githubusercontent.com/Altaieb-Mohammed/lab_2corse/master/inheritance_combined.csv"
df = load_data(url)


In [ ]:
#Выводим размер набора данных (число строк и столбцов).

                                                  #Показываем первые несколько строк для ознакомления с форматом и содержанием
print(f"Размер данных: {df.shape}")
print("Пример данных:")
print(df.head())


In [ ]:
#Выбираем только нужные числовые признаки для анализа.

#Заполняем пропущенные значения медианой каждого признака, чтобы избежать проблем с обучением моделей.
feature_columns = [
    'debts', 'bequests', 'wife', 'husband', 'sons', 'daughters',
    'father', 'mother', 'brothers', 'sisters'
]

df_selected = df[feature_columns].copy()
df_selected.fillna(df_selected.median(), inplace=True)


In [ ]:
#Создаём целевой признак high_debts:

#1 - если значение debts выше или равно медиане,

               # 0 - иначе.

                         #Это превращает задачу в бинарную классификацию.

median_debts = df_selected['debts'].median()
df_selected['high_debts'] = (df_selected['debts'] >= median_debts).astype(int)


In [ ]:
#Отделяем признаки X от целевого признака y.

#Убираем из признаков исходный debts, так как он напрямую связан с таргетом.



X = df_selected.drop(columns=['high_debts', 'debts'])
y = df_selected['high_debts']


In [ ]:
#Разбиваем данные на обучающую (80%) и тестовую (20%) выборки.

#Стратификация гарантирует сохранение пропорций классов в обеих выборках.


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
#Масштабируем признаки двумя способами:

          #StandardScaler - центрирует и масштабирует к среднему 0 и стандартному отклонению 1, подходит для f_classif.

                             #MinMaxScaler - масштабирует в диапазон , подходит для chi2.

                                          #Это необходимо для корректной работы методов отбора признаков.




scaler_std = StandardScaler()
X_train_scaled = scaler_std.fit_transform(X_train)
X_test_scaled = scaler_std.transform(X_test)

scaler_minmax = MinMaxScaler()
X_train_minmax = scaler_minmax.fit_transform(X_train)
X_test_minmax = scaler_minmax.transform(X_test)


In [ ]:
#Для каждого количества признаков от 1 до max_features выбираем лучшие признаки с помощью SelectKBest и функции оценки (f_classif или chi2).

#Обучаем RandomForestRegressor на выбранных признаках и оцениваем качество на тесте с помощью R².

#Строим график зависимости R² от числа признаков, чтобы увидеть, сколько признаков достаточно для хорошей модели.

#Возвращаем выбранные признаки и результаты для анализа.



def evaluate_feature_selection(score_func, score_func_name, X_train, X_test, y_train, y_test, feature_names, max_features=10):
    feature_counts = list(range(1, min(X_train.shape[1], max_features) + 1))
    scores = []
    selected_features_dict = {}

    for k in feature_counts:
        selector = SelectKBest(score_func, k=k)
        X_train_sel = selector.fit_transform(X_train, y_train)
        X_test_sel = selector.transform(X_test)

        model = RandomForestRegressor(random_state=42)
        model.fit(X_train_sel, y_train)
        y_pred = model.predict(X_test_sel)

        r2 = r2_score(y_test, y_pred)
        scores.append(r2)

        selected = [f for f, s in zip(feature_names, selector.get_support()) if s]
        selected_features_dict[k] = selected

    plt.figure(figsize=(10,6))
    plt.plot(feature_counts, scores, marker='o', label=score_func_name)
    plt.xlabel('Количество признаков')
    plt.ylabel('R² (коэффициент детерминации)')
    plt.title(f'Зависимость R² от количества признаков ({score_func_name})')
    plt.legend()
    plt.grid(True)
    plt.show()

    return feature_counts, scores, selected_features_dict


In [ ]:
#Создаём таблицу с результатами для обоих методов отбора: количество признаков, R² и выбранные признаки.

                                                  #Выводим первые 10 строк и сохраняем в CSV для удобного анализа.

results_df = pd.DataFrame({
    'Количество признаков': fc_counts,
    'R² (f_classif)': fc_scores,
    'Выбранные признаки (f_classif)': [", ".join(fc_features[k]) for k in fc_counts],
    'R² (chi2)': chi_scores,
    'Выбранные признаки (chi2)': [", ".join(chi_features[k]) for k in chi_counts],
})

print(results_df.head(10))
results_df.to_csv('feature_selection_results.csv', index=False)
print("Результаты сохранены в 'feature_selection_results.csv'")


In [ ]:
#Обучаем Random Forest на всех признаках (масштабированных StandardScaler).

#Получаем и визуализируем важность каждого признака - это помогает понять, какие признаки наиболее влиятельны для модели.



model_rf = RandomForestRegressor(random_state=42)
model_rf.fit(X_train_scaled, y_train)

importances = pd.Series(model_rf.feature_importances_, index=feature_names).sort_values(ascending=True)

plt.figure(figsize=(10,6))
importances.plot(kind='barh', color='teal')
plt.title('Важность признаков (Random Forest)')
plt.xlabel('Важность')
plt.show()
